### Categorize kanji

In [1]:
# imports
import pandas
import logging

from dataclasses import dataclass

logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s %(levelname)-4s %(message)s', datefmt='%m/%d %H:%M:%S')

# data classes
@dataclass        
class Kanji:
    rank: int
    name: str

In [75]:
# read excel file (with mapping)
df_kanji = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="MAIN")
df_kanji.columns = ["CHAR", "COMPONENTS1", "COMPONENTS2", "COMPONENTS3", "COMPONENTS4", "COMPONENTS5", "ON READING", "KUN READING", "KEYWORD", "SRL", "TYPE", "FREQ", "TAGS"]
df_kanji.fillna('', inplace=True)
df_kanji

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ,TAGS
0,一,,,,,,イチ、イツ,ひと・つ,one,5,STEM,10.800000,
1,二,,,,,,ニ,ふた・つ,two,5,MEAN,128.300000,
2,三,,,,,,サン,みっ・つ,three,5,MEAN,120.700000,
3,四,,,,,,シ,よっ・つ、よん、よ,four,5,MEAN,312.073333,
4,五,,,,,,ゴ,いつ・つ,five,5,MEAN,315.626667,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,気,气,㐅,,,,キ、ケ,,atmosphere,1,OTHER,64.000000,
1496,風,𠘨,䖝,,,,フウ、フ,かぜ,1 wind 2 style,1,OTHER,289.700000,
1497,乳,⺤,子,乚,,,ニュウ,ち、ちち,milk,1,OTHER,1067.000000,
1498,興,臼,同,ハ,,,コウ、キョウ,,interest,1,OTHER,649.600000,


In [4]:
df_keyword = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="keyword.list")
# df_keyword

In [5]:
df_stem = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="stem.list")
# df_stem

### Algorithm

In [81]:
random_row = df_kanji[df_kanji["CHAR"] == '精'].iloc[0]
random_row

CHAR                   精
COMPONENTS1            忄
COMPONENTS2            青
COMPONENTS3             
COMPONENTS4             
COMPONENTS5             
ON READING            セイ
KUN READING             
KEYWORD        mentality
SRL                    1
TYPE                  VR
FREQ               655.1
TAGS                    
Name: 33, dtype: object

In [82]:
categorization = {}
special_grp = '78 special'
other_grp = '77 other'
visual_grp = 'visual'

for grp in df_keyword["GROUP"].unique():
    categorization[grp] = []

for grp in df_stem["GROUP"].unique():
    categorization[grp] = []

categorization[other_grp] = []
categorization[special_grp] = []
categorization[visual_grp] = []

queue_categorization = {}

In [110]:
# first rule (check keyword)
def find_keyword(row):
    group = df_keyword[df_keyword["KEYWORD"] == row["KEYWORD"]]["GROUP"]
    if group.empty:
        return "none"
    else:
        return group.iloc[0]

def find_stem(row):
    group = df_stem[df_stem["STEM KANJI"] == row["CHAR"]]["GROUP"]
    if group.empty:
        return "none"
    else:
        return group.iloc[0]
    
def append_categorization(char, row, is_first):
    if is_first:
        if char in queue_categorization.keys():
            for ch in queue_categorization[char]:
                categorization[char].insert(0, ch)
            categorization[char].insert(0, row)
            del queue_categorization[char] 
        else:
            categorization[char].insert(0, row)
    else:
        if char in queue_categorization.keys():
            categorization[char].append(row)
            for ch in queue_categorization[char]:
                categorization[char].append(ch)
            del queue_categorization[char]
        else:
            categorization[char].append(row)

def find_cluster_1_2_components(component, row):
    return df_kanji[
        ((df_kanji["COMPONENTS1"] == row[component]) | 
         (df_kanji["COMPONENTS2"] == row[component])) 
        & (df_kanji["CHAR"] != row["CHAR"])
    ]

def find_cluster_components(component, row):
    return df_kanji[
        ((df_kanji["COMPONENTS2"] == row[component]) | 
         (df_kanji["COMPONENTS3"] == row[component]) |
         (df_kanji["COMPONENTS4"] == row[component]) |
         (df_kanji["COMPONENTS5"] == row[component])) 
        & (df_kanji["CHAR"] != row["CHAR"])
    ]

def find_cluster_all_components(component, row):
    return df_kanji[
        ((df_kanji["COMPONENTS1"] == row[component]) | 
        (df_kanji["COMPONENTS2"] == row[component]) | 
        (df_kanji["COMPONENTS3"] == row[component]) |
        (df_kanji["COMPONENTS4"] == row[component]) |
        (df_kanji["COMPONENTS5"] == row[component]))
        & (df_kanji["CHAR"] != row["CHAR"])
    ]

def find_onyomi(random_row, vr_cluster):
    vr_crowns = random_row["ON READING"].split("、")
    onyomi = vr_cluster[vr_cluster["ON READING"].isin(vr_crowns)]
    if onyomi.empty:
#         print("4. rule - 3rd condition")
        vr_third_cluster = df_kanji[df_kanji["CHAR"] == random_row["COMPONENTS2"]]
        if len(vr_third_cluster.index) > 0:
            if len(vr_third_cluster.index) > 1:
#                 print("kanji > 1")
                max_srl_kanji = vr_third_cluster[vr_third_cluster["SRL"] == vr_third_cluster["SRL"].max()].iloc[0]
                random_row["TYPE"] = "VR"

                if max_srl_kanji["CHAR"] in queue_categorization.keys():
                    queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
                else:
                    queue_categorization[max_srl_kanji["CHAR"]] = []
                    queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
            else:
#                 print("kanji = 1")
                random_row["TYPE"] = "VR"
                max_srl_kanji = vr_third_cluster.iloc[0]
                if max_srl_kanji["CHAR"] in queue_categorization.keys():
                    queue_categorization[vr_third_cluster.iloc[0]["CHAR"]].append(random_row)
                else:
                    queue_categorization[vr_third_cluster.iloc[0]["CHAR"]] = []
                    queue_categorization[vr_third_cluster.iloc[0]["CHAR"]].append(random_row)
        else:
            fifth_rule(random_row)
    else:
        if len(onyomi.index) > 1:
#             print("kanji > 1")
            max_srl_kanji = onyomi[onyomi["SRL"] == onyomi["SRL"].max()].iloc[0]
            random_row["TYPE"] = "VR"

            if max_srl_kanji["CHAR"] in queue_categorization.keys():
                queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
            else:
                queue_categorization[max_srl_kanji["CHAR"]] = []
                queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
        else:
#             print("kanji = 1")
            random_row["TYPE"] = "VR"
            max_srl_kanji = onyomi.iloc[0]
            if max_srl_kanji["CHAR"] in queue_categorization.keys():
                queue_categorization[onyomi.iloc[0]["CHAR"]].append(random_row)
            else:
                queue_categorization[onyomi.iloc[0]["CHAR"]] = []
                queue_categorization[onyomi.iloc[0]["CHAR"]].append(random_row)

def seventh_rule(random_row):
#     print("TODO 7. rule")
    append_categorization(other_grp, random_row, False)
                
def sixth_rule(random_row):
#     print("6. rule")
    vr_cluster_1_2 = pandas.concat([
        find_cluster_1_2_components("COMPONENTS1", random_row), 
        find_cluster_1_2_components("COMPONENTS2", random_row)
    ])
    if vr_cluster_1_2.empty:
        seventh_rule(random_row)
    else:
        if len(vr_cluster_1_2.index) > 1:
            append_categorization(visual_grp, random_row, False)
        else:
            random_row["TYPE"] = "VISUAL"
            max_srl_kanji = vr_cluster_1_2.iloc[0]
            if max_srl_kanji["CHAR"] in queue_categorization.keys():
                queue_categorization[vr_cluster_1_2.iloc[0]["CHAR"]].append(random_row)
            else:
                queue_categorization[vr_cluster_1_2.iloc[0]["CHAR"]] = []
                queue_categorization[vr_cluster_1_2.iloc[0]["CHAR"]].append(random_row)
                
def fifth_rule(random_row):
#     print("5. rule")
    group_stem = df_stem[
        (df_stem["STEM KANJI"] == random_row["COMPONENTS1"]) |
        (df_stem["STEM KANJI"] == random_row["COMPONENTS2"]) |
        (df_stem["STEM KANJI"] == random_row["COMPONENTS3"])
    ]["GROUP"]
    if group_stem.empty:
        sixth_rule(random_row)
    else:
        if (random_row["SRL"] == 1):
            random_row["TYPE"] = "FORM"
        else:
            random_row["TYPE"] = "MEAN"
        if len(group_stem) == 1: 
            append_categorization(group_stem.iloc[0], random_row, False)
        else:
            print("more stems TODO")
                
def fourth_rule(random_row):
#     print("4. rule")
    vr_cluster = pandas.concat([
        find_cluster_components("COMPONENTS2", random_row), 
        find_cluster_components("COMPONENTS3", random_row), 
        find_cluster_components("COMPONENTS4", random_row), 
        find_cluster_components("COMPONENTS5", random_row)
    ])
    if vr_cluster.empty:
        vr_all_cluster = pandas.concat([
            find_cluster_all_components("COMPONENTS1", random_row),
            find_cluster_all_components("COMPONENTS2", random_row),
            find_cluster_all_components("COMPONENTS3", random_row),
            find_cluster_all_components("COMPONENTS4", random_row),
            find_cluster_all_components("COMPONENTS5", random_row)
        ])
        if vr_all_cluster.empty:
            fifth_rule(random_row)
        else:
#             print("vr all cluster")
            find_onyomi(random_row, vr_all_cluster)
    else:
#         print("vr clusters")
        find_onyomi(random_row, vr_cluster)
        
def categorize_kanji(random_row):
    first_rule = find_keyword(random_row)
    second_rule = find_stem(random_row)
    if (first_rule != "none"):
#         print("1. rule")
        if (random_row["TYPE"] == "MEAN"):
            append_categorization(first_rule, random_row, False)
        else:
            if (random_row["TYPE"] == "SPECIAL"):
                append_categorization(special_grp, random_row, False)
            else:
                if (random_row["TYPE"] == "OTHER"):
                    append_categorization(other_grp, random_row, False)
                else:
                    print("ERROR: missing grp")
    else:
        if (second_rule != "none"):
#             print("2. rule")
            if (random_row["TYPE"] == "STEM"):
                append_categorization(second_rule, random_row, True)
            else:
                print("ERROR: missing rule")
        else:
            components = df_kanji[(df_kanji["COMPONENTS1"] == random_row["CHAR"]) | (df_kanji["COMPONENTS2"] == random_row["CHAR"])]
#             print("components count: " + str(len(components.index)))
            if components.empty:
                fourth_rule(random_row)
            else:
#                 print("3. rule")
                vr_crowns = random_row["ON READING"].split("、")
#                 print(vr_crowns)
                onyomi = components[components["ON READING"].isin(vr_crowns)]
                if onyomi.empty:
#                     print("onyomi empty")
                    fourth_rule(random_row)
                else:
#                     print("3. rule a) b)")
                    if len(onyomi.index) > 1:
#                         print("kanji > 1")
#                         print(onyomi)
                        max_srl_kanji = onyomi[onyomi["SRL"] == onyomi["SRL"].max()].iloc[0]
                        random_row["TAG"] = "CROWN_TAG"
                        random_row["TYPE"] = "VR"

                        if max_srl_kanji["CHAR"] in queue_categorization.keys():
                            queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
                        else:
                            queue_categorization[max_srl_kanji["CHAR"]] = []
                            queue_categorization[max_srl_kanji["CHAR"]].append(random_row)
                    else:
#                         print("kanji = 1")
                        random_row["TAG"] = "CROWN_TAG"
                        random_row["TYPE"] = "VR"
                        max_srl_kanji = onyomi.iloc[0]
                        if max_srl_kanji["CHAR"] in queue_categorization.keys():
                            queue_categorization[onyomi.iloc[0]["CHAR"]].append(random_row)
                        else:
                            queue_categorization[onyomi.iloc[0]["CHAR"]] = []
                            queue_categorization[onyomi.iloc[0]["CHAR"]].append(random_row)

In [84]:
categorize_kanji(random_row)

In [111]:
categorization = {}
special_grp = '78 special'
other_grp = '77 other'
visual_grp = 'visual'

for grp in df_keyword["GROUP"].unique():
    categorization[grp] = []

for grp in df_stem["GROUP"].unique():
    categorization[grp] = []

categorization[other_grp] = []
categorization[special_grp] = []
categorization[visual_grp] = []

queue_categorization = {}

# for i in range(100):

for i in range(len(df_kanji)):
    kanji = df_kanji.loc[i]
    print(kanji["CHAR"])
    categorize_kanji(kanji)

一
二
三
四
五
六
七
八
九
十
百
千
万
億
憶
兆
眺
挑
父
母
兄
況
競
弟
第
姉
市
妹
黒
紫
青
清
請
精
情
静
争
紅
工
項
攻
功
貢
赤
緑
黄
横
白
泊
迫
拍
日
旧
曜
昨
作
晴
暗
暖
暇
昇
弁
飛
景
京
就
量
星
生
性
姓
易
是
者
老
考
孝
音
響
郷
香
書
普
並
替
月
脱
服
肌
脚
却
腰
要
腹
複
復
覆
腕
脇
胸
臓
蔵
脳
悩
肝
干
刊
岸
幹
乾
韓
胞
包
抱
砲
脈
肥
勝
膨
背
骨
肩
有
脅
協
火
炎
焼
燃
然
煙
灯
畑
爆
暴
災
熱
煮
焦
蒸
承
了
無
舞
黙
照
昭
召
招
紹
熟
水
氷
洋
海
悔
毎
津
波
破
流
瀬
湾
江
浦
浜
兵
港
湖
沢
択
沼
泥
河
川
州
可
何
荷
歌
源
原
湯
池
地
施
温
湿
溶
容
滑
浴
欲
泳
永
久
浮
浸
侵
寝
漏
沈
潜
浅
銭
深
没
治
派
渉
歩
沿
法
去
準
決
済
剤
減
漁
魚
渡
度
漢
沖
滅
洗
先
汚
液
濃
農
測
則
側
注
駐
満
両
円
混
添
油
由
消
浄
澄
演
浪
良
渋
泣
涙
涼
活
括
激
酒
配
酔
粋
酷
告
酸
木
林
森
樹
桜
桃
松
公
梅
柳
植
殖
根
核
枝
支
棒
奉
柱
枠
棚
橋
校
交
絞
効
村
極
権
機
械
戒
模
札
標
票
株
枚
杯
様
概
既
条
柔
栗
集
楽
乗
染
査
社
祈
神
申
伸
祖
狙
組
祝
礼
祉
止
歯
初
裕
袖
裸
金
銀
銅
同
洞
筒
鉄
失
鉱
広
銃
充
鎖
針
鋭
釣
鈴
令
冷
齢
銘
名
録
土
場
域
境
鏡
垣
壇
塚
堀
城
成
誠
盛
壊
懐
埋
塩
均
声
売
喜
塗
壁
癖
王
理
里
現
珍
皇
聖
望
美
着
養
表
面
素
毒
麦
人
例
個
固
古
故
俺
僕
撲
業
偽
為
倒
到
債
他
佐
左
右
差
借
任
妊
働
動
仕
士
位
係
系
休
住
信
僧
増
贈
憎
層
仏
仁
健
建
優
偉
違
像
象
体
本
傷
供
共
備
仮
便
使
史
丈
俗
谷
似
以
低
抵
邸
底
価
値
置
直
催
件
倍
保
修
候
併
停
亭
依
衣
存
在
行
来
往
主
徒
後
前
得
徳
律
術
述
街
衛
衝
微
徴
徹
撤
女
娘
婚
嫁


In [112]:
print("categorization")
for key in categorization:
    print(key)
    for ch in categorization[key]:
        print(ch["CHAR"])
        
print("queue_categorization")
for key in queue_categorization:
    print("key: ", key)
    for ch in queue_categorization[key]:
        print(ch["CHAR"])
    print("----")

categorization
1 numbers
一
二
三
四
五
六
七
八
九
十
百
千
万
億
兆
2 family
父
母
兄
弟
姉
妹
3 colors
黒
紫
青
紅
赤
緑
黄
白
31 cardinals
北
南
西
東
32 season of the year
夏
秋
冬
33 parts of the day
朝
午
昼
晩
夜
77 other
上
下
互
世
甘
気
風
乳
興
疑
4 Sunday
日
是
5 Monday
月
脈
6 Tuesday
火
滅
7 flame
熱
8 Wednesday
水
9 alcohol
酒
10 Thursday
木
11 Friday
金
12 Saturday
土
域
13 human
人
14 woman
女
15 child
子
16 ear
耳
17 eye
目
18 to see
見
19 mouth
口
20 tongue
舌
21 talk
言
論
22 beard
形
23 heart
心
24 arm
手
25 leg
足
26 kneel
印
27 stand
立
28 run
走
越
29 route
途
30 direction
方
32 seasons of the year
春
34 car
車
35 ship
船
36 rain
雨
37 gate
門
38 roof A
宗
39 roof B
玄
40 roof C
居
41 roof D
企
42 temple
寺
43 insect
虫
44 dog
犬
45 hound
犯
46 sheep
羊
47 cow
牛
48 horse
馬
49 bird
唯
50 feather
羽
51 field
田
52 mound
隊
53 grass
草
54 thread
糸
55 bamboo
竹
56 rice
米
57 grain
私
58 stone
石
59 mountain
山
60 dishes
皿
61 cloth
帳
62 to eat
食
63 tiny
寸
64 big
大
65 sword
剣
66 to strike
改
67 bow
弓
68 arrow
矢
69 pike
設
70 again
又
71 halberd
我
72 axe
析
73 horn
角
74 dress
装

In [ ]:
print("count categorization")
for key in categorization:
    print(key)
    i = 0
    for ch in categorization[key]: